In [5]:
# Connect to GDrive to access image files and labels
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
%cd '/content/gdrive/MyDrive/__Shared/Andrew/working_directory'
%pwd

/content/gdrive/.shortcut-targets-by-id/1Bhg_-iEsxa3Aq4v-SLxPVOpKrY5TpKaq/Andrew/working_directory


'/content/gdrive/.shortcut-targets-by-id/1Bhg_-iEsxa3Aq4v-SLxPVOpKrY5TpKaq/Andrew/working_directory'

In [7]:
import os

from data_processing.data_preprocess import get_json_dict, get_json_img_list, slice_image, adjust_coco_annotations, slice_batch

In [4]:
### testing on heli for now

# raw_img_dir = '/Users/gabriellemelli/Library/CloudStorage/GoogleDrive-gabbymelli@gmail.com/My Drive/Work Stuff/Freelance Work/Andrew/working_directory/data_sources'
raw_img_dir = 'data_sources'

heli_subdir = 'white_sands_heli/xstrata/images'

img_dir = os.path.join(raw_img_dir, heli_subdir)
raw_img_file_list = os.listdir(img_dir)
anno_file = os.path.join(raw_img_dir, 'white_sands_heli/xstrata/labels/segmentation_xstrata_labels.json')

output_dir = 'output/test_img_slice'

In [5]:
test = slice_batch(img_dir, anno_file, (500, 500), output_dir)

img_path: AnnaWhiteSandsAerials-Vertical--353-IMG_3240.JPG.jpg
full_path: data_sources/white_sands_heli/xstrata/images/AnnaWhiteSandsAerials-Vertical--353-IMG_3240.JPG.jpg
orig_img_id: 1
original_img_id: 1
orig_img_annots: [{'id': 0, 'iscrowd': 0, 'image_id': 1, 'category_id': 1, 'segmentation': [[502.43316966721187, 669.3289689034366, 484.6262956901252, 652.5695581014726, 485.6737588652479, 630.5728314238949, 483.57883251500243, 612.7659574468082, 484.6262956901252, 593.9116202945987, 494.0534642662299, 572.9623567921437, 498.2433169667209, 549.9181669394433, 515.0027277686848, 539.4435351882157, 520.2400436442987, 560.3927986906707, 530.7146753955261, 590.7692307692305, 536.9994544462626, 619.0507364975447, 541.1893071467537, 645.2373158756134, 548.5215493726129, 660.9492635024546, 552.7114020731038, 674.5662847790503, 552.7114020731038, 686.0883797054006, 547.4740861974901, 693.4206219312598, 544.3316966721219, 699.7054009819964, 539.0943807965081, 705.9901800327328, 529.66721222040

In [6]:
import pprint

pprint.pp(test)

Streaming output truncated to the last 5000 lines.
                                    814.8608837970536,
                                    -429.5908346972177,
                                    847.5613747954169,
                                    -419.8690671031097,
                                    875.8428805237311,
                                    -403.96072013093294,
                                    889.9836333878882,
                                    -383.63338788870715,
                                    922.6841243862515,
                                    -357.11947626841254,
                                    946.5466448445167,
                                    -320.0000000000001,
                                    984.5499181669388,
                                    -292.6022913256957,
                                    1010.1800327332236,
                                    -274.9263502454993,
                                    1040.2291325695576,
 

In [8]:
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

def plot_annotations(image_path, annotations, category_map):
    # Load the image
    img = np.array(Image.open(image_path))

    # Create a plot
    fig, ax = plt.subplots(1, figsize=(12, 12))
    ax.imshow(img)

    # Iterate over annotations and plot bounding boxes and masks
    for annotation in annotations:
        # Plot bounding box (if available)
        if 'bbox' in annotation:
            bbox = annotation['bbox']
            rect = patches.Rectangle((bbox[0], bbox[1]), bbox[2], bbox[3], linewidth=2, edgecolor='r', facecolor='none')
            ax.add_patch(rect)

        # Plot segmentation mask (if available)
        if 'segmentation' in annotation:
            for seg in annotation['segmentation']:
                poly = np.array(seg).reshape((int(len(seg)/2), 2))
                polygon = patches.Polygon(poly, linewidth=2, edgecolor='blue', facecolor='none')
                ax.add_patch(polygon)

        # Add the category name
        category_id = annotation['category_id']
        category_name = category_map[category_id]
        plt.text(bbox[0], bbox[1] - 5, category_name, color='white', fontsize=12, bbox=dict(facecolor='red', alpha=0.5))

    plt.show()

def get_image_annotations(image_id, coco_annotations):
    # Filter annotations corresponding to the given image_id
    return [ann for ann in coco_annotations if ann['image_id'] == image_id]

def load_category_map(coco_data):
    # Create a mapping from category_id to category name
    return {category['id']: category['name'] for category in coco_data['categories']}

In [10]:
# Example usage
# coco_annotation_path = '/Users/gabriellemelli/Desktop/test_img_slice/new_labels.json'  # Path to your new COCO annotations
# image_dir = '/Users/gabriellemelli/Desktop/test_img_slice/images'  # Directory where your sliced images are stored
coco_annotation_path = 'output/test_img_slice/new_labels.json'  # Path to your new COCO annotations
image_dir = 'output/test_img_slice/images' # Directory where your sliced images are stored


# Load the COCO annotations
with open(coco_annotation_path) as f:
    coco_data = json.load(f)

# Create a category map from category_id to category name
category_map = load_category_map(coco_data)

# Iterate over the images in the COCO dataset
for image_info in coco_data['images']:
    image_id = image_info['id']
    image_filename = image_info['file_name']
    image_path = f"{image_dir}/{image_filename}"

    # Get annotations for this image
    annotations = get_image_annotations(image_id, coco_data['annotations'])

    # Plot the image with overlaid annotations
    plot_annotations(image_path, annotations, category_map)


Output hidden; open in https://colab.research.google.com to view.

In [2]:
import os
os.getcwd()

'/content'